**Baseline was taken from the beautiful** [Enefit Generic Notebook](https://www.kaggle.com/code/greysky/enefit-generic-notebook) 🙏

In [1]:
import os
import gc
import pickle

import numpy as np
import pandas as pd
import polars as pl
import plotly.express as px

from sklearn.ensemble import VotingRegressor

import lightgbm as lgb
from catboost import CatBoostRegressor

# Read Data

In [3]:
root = "Dataset/predict-energy-behavior-of-prosumers"

data_cols = ['target', 'county', 'is_business', 'product_type', 'is_consumption', 'datetime', 'row_id']
client_cols = ['product_type', 'county', 'eic_count', 'installed_capacity', 'is_business', 'date']
gas_prices_cols = ['forecast_date', 'lowest_price_per_mwh', 'highest_price_per_mwh']
electricity_prices_cols = ['forecast_date', 'euros_per_mwh']
forecast_weather_cols = ['latitude', 'longitude', 'hours_ahead', 'temperature', 'dewpoint', 'cloudcover_high', 'cloudcover_low', 'cloudcover_mid', 'cloudcover_total', '10_metre_u_wind_component', '10_metre_v_wind_component', 'forecast_datetime', 'direct_solar_radiation', 'surface_solar_radiation_downwards', 'snowfall', 'total_precipitation']
historical_weather_cols = ['datetime', 'temperature', 'dewpoint', 'rain', 'snowfall', 'surface_pressure','cloudcover_total','cloudcover_low','cloudcover_mid','cloudcover_high','windspeed_10m','winddirection_10m','shortwave_radiation','direct_solar_radiation','diffuse_radiation','latitude','longitude']
location_cols = ['longitude', 'latitude', 'county']
target_cols = ['target', 'county', 'is_business', 'product_type', 'is_consumption', 'datetime']

In [4]:
df_data = pl.read_csv(os.path.join(root, "train.csv"), columns=data_cols, try_parse_dates=True)
df_client = pl.read_csv(os.path.join(root, "client.csv"), columns=client_cols, try_parse_dates=True)
df_gas_prices = pl.read_csv(os.path.join(root, "gas_prices.csv"), columns=gas_prices_cols, try_parse_dates=True)
df_electricity_prices = pl.read_csv(os.path.join(root, "electricity_prices.csv"), columns=electricity_prices_cols, try_parse_dates=True)
df_forecast_weather = pl.read_csv(os.path.join(root, "forecast_weather.csv"), columns=forecast_weather_cols, try_parse_dates=True)
df_historical_weather = pl.read_csv(os.path.join(root, "historical_weather.csv"), columns=historical_weather_cols, try_parse_dates=True)
df_weather_station_to_county_mapping = pl.read_csv(os.path.join(root, "weather_station_to_county_mapping.csv"), columns=location_cols, try_parse_dates=True)
df_target = df_data.select(target_cols)

schema_data = df_data.schema
schema_client = df_client.schema
schema_gas  = df_gas_prices.schema
schema_electricity = df_electricity_prices.schema
schema_forecast = df_forecast_weather.schema
schema_historical = df_historical_weather.schema
schema_target = df_target.schema

# Features Generation

Polars library is faster than Pandas, which explains the choice.

In [5]:
def generate_features(
        df_data, 
        df_client, 
        df_gas_prices, 
        df_electricity_prices, 
        df_forecast_weather, 
        df_historical_weather, 
        df_weather_station_to_county_mapping, 
        df_target
):
    df_data = (
        df_data
        .with_columns(
            pl.col("datetime").cast(pl.Date).alias("date"),
        )
    )
    
    df_gas_prices = (
        df_gas_prices
        .rename({"forecast_date": "date"})
    )
    
    df_electricity_prices = (
        df_electricity_prices
        .rename({"forecast_date": "datetime"})
    )
    
    df_weather_station_to_county_mapping = (
        df_weather_station_to_county_mapping
        .with_columns(
            pl.col("latitude").cast(pl.datatypes.Float32),
            pl.col("longitude").cast(pl.datatypes.Float32)
        )
    )
    
    # sum of all product_type targets related to ["datetime", "county", "is_business", "is_consumption"]
    df_target_all_type_sum = (
        df_target
        .group_by(["datetime", "county", "is_business", "is_consumption"]).sum()
        .drop("product_type")
    )
    
    df_forecast_weather = (
        df_forecast_weather
        .rename({"forecast_datetime": "datetime"})
        .filter(pl.col("hours_ahead") >= 24) # we don't need forecast for today
        .with_columns(
            pl.col("latitude").cast(pl.datatypes.Float32),
            pl.col("longitude").cast(pl.datatypes.Float32),
            # datetime for forecast in a different timezone
            pl.col('datetime').dt.replace_time_zone(None).cast(pl.Datetime("us"))
        )
        .join(df_weather_station_to_county_mapping, how="left", on=["longitude", "latitude"])
        .drop("longitude", "latitude")
    )
    
    df_historical_weather = (
        df_historical_weather
        .with_columns(
            pl.col("latitude").cast(pl.datatypes.Float32),
            pl.col("longitude").cast(pl.datatypes.Float32),
#            pl.col("datetime") + pl.duration(hours=37)
        )
        .join(df_weather_station_to_county_mapping, how="left", on=["longitude", "latitude"])
        .drop("longitude", "latitude")
    )
    
    # creating average forecast characteristics for all weather stations
    df_forecast_weather_date = (
        df_forecast_weather
        .group_by("datetime").mean()
        .drop("county")
    )
    
    # creating average forecast characteristics for weather stations related to county
    df_forecast_weather_local = (
        df_forecast_weather
        .filter(pl.col("county").is_not_null())
        .group_by("county", "datetime").mean()
    )
    
    # creating average historical characteristics for all weather stations
    df_historical_weather_date = (
        df_historical_weather
        .group_by("datetime").mean()
        .drop("county")
    )
    
    # creating average historical characteristics for weather stations related to county
    df_historical_weather_local = (
        df_historical_weather
        .filter(pl.col("county").is_not_null())
        .group_by("county", "datetime").mean()
    )
    
    df_data = (
        df_data
        # pl.duration(days=1) shifts datetime to join lag features (usually we join last available values)
        .join(df_gas_prices.with_columns((pl.col("date") + pl.duration(days=1)).cast(pl.Date)), on="date", how="left")
        .join(df_client.with_columns((pl.col("date") + pl.duration(days=2)).cast(pl.Date)), on=["county", "is_business", "product_type", "date"], how="left")
        .join(df_electricity_prices.with_columns(pl.col("datetime") + pl.duration(days=1)), on="datetime", how="left")
        
        # lag forecast_weather features (24 hours * days)
        .join(df_forecast_weather_date, on="datetime", how="left", suffix="_fd")
        .join(df_forecast_weather_local, on=["county", "datetime"], how="left", suffix="_fl")
        .join(df_forecast_weather_date.with_columns(pl.col("datetime") + pl.duration(days=7)), on="datetime", how="left", suffix="_fd_7d")
        .join(df_forecast_weather_local.with_columns(pl.col("datetime") + pl.duration(days=7)), on=["county", "datetime"], how="left", suffix="_fl_7d")

        # lag historical_weather features (24 hours * days)
        .join(df_historical_weather_date.with_columns(pl.col("datetime") + pl.duration(days=2)), on="datetime", how="left", suffix="_hd_2d")
        .join(df_historical_weather_local.with_columns(pl.col("datetime") + pl.duration(days=2)), on=["county", "datetime"], how="left", suffix="_hl_2d")
        .join(df_historical_weather_date.with_columns(pl.col("datetime") + pl.duration(days=7)), on="datetime", how="left", suffix="_hd_7d")
        .join(df_historical_weather_local.with_columns(pl.col("datetime") + pl.duration(days=7)), on=["county", "datetime"], how="left", suffix="_hl_7d")
        
        # lag target features (24 hours * days)
        .join(df_target.with_columns(pl.col("datetime") + pl.duration(days=2)).rename({"target": "target_1"}), on=["county", "is_business", "product_type", "is_consumption", "datetime"], how="left")
        .join(df_target.with_columns(pl.col("datetime") + pl.duration(days=3)).rename({"target": "target_2"}), on=["county", "is_business", "product_type", "is_consumption", "datetime"], how="left")
        .join(df_target.with_columns(pl.col("datetime") + pl.duration(days=4)).rename({"target": "target_3"}), on=["county", "is_business", "product_type", "is_consumption", "datetime"], how="left")
        .join(df_target.with_columns(pl.col("datetime") + pl.duration(days=5)).rename({"target": "target_4"}), on=["county", "is_business", "product_type", "is_consumption", "datetime"], how="left")
        .join(df_target.with_columns(pl.col("datetime") + pl.duration(days=6)).rename({"target": "target_5"}), on=["county", "is_business", "product_type", "is_consumption", "datetime"], how="left")
        .join(df_target.with_columns(pl.col("datetime") + pl.duration(days=7)).rename({"target": "target_6"}), on=["county", "is_business", "product_type", "is_consumption", "datetime"], how="left")
        .join(df_target.with_columns(pl.col("datetime") + pl.duration(days=14)).rename({"target": "target_7"}), on=["county", "is_business", "product_type", "is_consumption", "datetime"], how="left")
        
        .join(df_target_all_type_sum.with_columns(pl.col("datetime") + pl.duration(days=2)).rename({"target": "target_1"}), on=["county", "is_business", "is_consumption", "datetime"], suffix="_all_type_sum", how="left")
        .join(df_target_all_type_sum.with_columns(pl.col("datetime") + pl.duration(days=3)).rename({"target": "target_2"}), on=["county", "is_business", "is_consumption", "datetime"], suffix="_all_type_sum", how="left")
        .join(df_target_all_type_sum.with_columns(pl.col("datetime") + pl.duration(days=7)).rename({"target": "target_6"}), on=["county", "is_business", "is_consumption", "datetime"], suffix="_all_type_sum", how="left")
        .join(df_target_all_type_sum.with_columns(pl.col("datetime") + pl.duration(days=14)).rename({"target": "target_7"}), on=["county", "is_business", "is_consumption", "datetime"], suffix="_all_type_sum", how="left")
        
        
        .with_columns(
            pl.col("datetime").dt.ordinal_day().alias("dayofyear"),
            pl.col("datetime").dt.hour().alias("hour"),
            pl.col("datetime").dt.day().alias("day"),
            pl.col("datetime").dt.weekday().alias("weekday"),
            pl.col("datetime").dt.month().alias("month"),
            pl.col("datetime").dt.year().alias("year"),
        )
        
        .with_columns(
            pl.concat_str("county", "is_business", "product_type", "is_consumption", separator="_").alias("segment"),
        )
        
        # cyclical features encoding https://towardsdatascience.com/cyclical-features-encoding-its-about-time-ce23581845ca
        .with_columns(
            (np.pi * pl.col("dayofyear") / 183).sin().alias("sin(dayofyear)"),
            (np.pi * pl.col("dayofyear") / 183).cos().alias("cos(dayofyear)"),
            (np.pi * pl.col("hour") / 12).sin().alias("sin(hour)"),
            (np.pi * pl.col("hour") / 12).cos().alias("cos(hour)"),
        )
        
        .with_columns(
            pl.col(pl.Float64).cast(pl.Float32),
        )
        
        .drop("date", "datetime", "hour", "dayofyear")
    )
    
    return df_data

In [6]:
def to_pandas(X, y=None):
    cat_cols = ["county", "is_business", "product_type", "is_consumption", "segment"]
    
    if y is not None:
        df = pd.concat([X.to_pandas(), y.to_pandas()], axis=1)
    else:
        df = X.to_pandas()    
    
    df = df.set_index("row_id")
    df[cat_cols] = df[cat_cols].astype("category")
    
    df["target_mean"] = df[[f"target_{i}" for i in range(1, 7)]].mean(1)
    df["target_std"] = df[[f"target_{i}" for i in range(1, 7)]].std(1)
    df["target_ratio"] = df["target_6"] / (df["target_7"] + 1e-3)
    
    return df

In [7]:
df_data, y = df_data.drop("target"), df_data.select("target")

df_train_features = generate_features(
    df_data, 
    df_client, 
    df_gas_prices, 
    df_electricity_prices, 
    df_forecast_weather, 
    df_historical_weather, 
    df_weather_station_to_county_mapping, 
    df_target
)

df_train_features = to_pandas(df_train_features, y)
# a little proportion of target values are null
df_train_features = df_train_features[df_train_features['target'].notnull()]

# filter old data
df_train_features = df_train_features[df_train_features.year >= 2022]

## Optuna

In [11]:
import optuna
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

In [21]:
import optuna
import lightgbm as lgb
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split

def objective(trial):
    # 하이퍼파라미터 범위 설정
    param = {
        'objective': 'regression',
        'metric': 'rmse',
        'n_estimators': trial.suggest_int('n_estimators', 500, 1500),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.1),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.7, 1.0),
        'colsample_bynode': trial.suggest_float('colsample_bynode', 0.5, 1.0),
        'lambda_l1': trial.suggest_float('lambda_l1', 0, 5),
        'lambda_l2': trial.suggest_float('lambda_l2', 0, 5),
        'max_depth': trial.suggest_int('max_depth', 5, 15)
    }

    # 데이터 분할
    mask_con = df_train_features['is_consumption'] == 1
    mask_pro = df_train_features['is_consumption'] == 0
    X_train_con, X_test_con, y_train_con, y_test_con = train_test_split(df_train_features[mask_con].drop(columns=["target"]), df_train_features[mask_con]["target"], test_size=0.2, random_state=42)
    X_train_pro, X_test_pro, y_train_pro, y_test_pro = train_test_split(df_train_features[mask_pro].drop(columns=["target"]), df_train_features[mask_pro]["target"], test_size=0.2, random_state=42)

    # 모델 훈련
    model_consumption = lgb.LGBMRegressor(**param)
    model_production = lgb.LGBMRegressor(**param)

    model_consumption.fit(X_train_con, y_train_con, eval_set=[(X_test_con, y_test_con)])
    model_production.fit(X_train_pro, y_train_pro, eval_set=[(X_test_pro, y_test_pro)])

    # 모델 성능 평가
    preds_consumption = model_consumption.predict(X_test_con)
    preds_production = model_production.predict(X_test_pro)
    mae_con = mean_absolute_error(y_test_con, preds_consumption)
    mae_pro = mean_absolute_error(y_test_pro, preds_production)

    # 두 모델의 평균 MAE 반환
    # MAE로 수정해서 할 것.
    return (mae_con + mae_pro) / 2


In [22]:
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=100, show_progress_bar=True)

print('Number of finished trials:', len(study.trials))
print('Best trial:', study.best_trial.params)


[I 2023-12-21 19:09:39,114] A new study created in memory with name: no-name-ebf5f77b-2a14-4a58-a7a5-d3b536ca2e07
  0%|          | 0/100 [00:00<?, ?it/s]

[LightGBM] [Warning] lambda_l1 is set=4.506308420266135, reg_alpha=0.0 will be ignored. Current value: lambda_l1=4.506308420266135
[LightGBM] [Warning] lambda_l2 is set=4.21685548320606, reg_lambda=0.0 will be ignored. Current value: lambda_l2=4.21685548320606
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] lambda_l1 is set=4.506308420266135, reg_alpha=0.0 will be ignored. Current value: lambda_l1=4.506308420266135
[LightGBM] [Warning] lambda_l2 is set=4.21685548320606, reg_lambda=0.0 will be ignored. Current value: lambda_l2=4.21685548320606
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.117387 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 31734
[LightGBM] [Info] Num

Best trial: 0. Best value: 26.4116:   1%|          | 1/100 [01:38<2:42:02, 98.21s/it]

[I 2023-12-21 19:11:17,321] Trial 0 finished with value: 26.411598388124492 and parameters: {'n_estimators': 1453, 'learning_rate': 0.06814461950338485, 'colsample_bytree': 0.8334858583775225, 'colsample_bynode': 0.7278770475028239, 'lambda_l1': 4.506308420266135, 'lambda_l2': 4.21685548320606, 'max_depth': 11}. Best is trial 0 with value: 26.411598388124492.
[LightGBM] [Warning] lambda_l1 is set=1.1215646816628828, reg_alpha=0.0 will be ignored. Current value: lambda_l1=1.1215646816628828
[LightGBM] [Warning] lambda_l2 is set=3.666749888134677, reg_lambda=0.0 will be ignored. Current value: lambda_l2=3.666749888134677
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] lambda_l1 is set=1.1215646816628828, reg_alpha=0.0 will be ignored. Current value: lambda_l1=1.1215646816628828
[LightGBM] [Warning] lambda_l2 is set=3.666749888134677, reg_lambda=0.0 will be ignored. Current value: lambda

Best trial: 0. Best value: 26.4116:   2%|▏         | 2/100 [02:36<2:02:13, 74.84s/it]

...

[I 2023-12-21 21:33:34,221] Trial 97 finished with value: 25.146901287362162 and parameters: {'n_estimators': 1437, 'learning_rate': 0.09322402509010909, 'colsample_bytree': 0.9914866251512339, 'colsample_bynode': 0.8457458628593397, 'lambda_l1': 4.068863768305403, 'lambda_l2': 0.9160032441370413, 'max_depth': 12}. Best is trial 81 with value: 24.675671423250755.
[LightGBM] [Warning] lambda_l1 is set=2.8478357976652857, reg_alpha=0.0 will be ignored. Current value: lambda_l1=2.8478357976652857
[LightGBM] [Warning] lambda_l2 is set=1.3445067021941788, reg_lambda=0.0 will be ignored. Current value: lambda_l2=1.3445067021941788
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] lambda_l1 is set=2.8478357976652857, reg_alpha=0.0 will be ignored. Current value: lambda_l1=2.8478357976652857
[LightGBM] [Warning] lambda_l2 is set=1.3445067021941788, reg_lambda=0.0 will be ignored. Current value:

Best trial: 81. Best value: 24.6757:  99%|█████████▉| 99/100 [2:25:35<01:33, 93.93s/it]

[I 2023-12-21 21:35:14,557] Trial 98 finished with value: 27.29576019212179 and parameters: {'n_estimators': 1499, 'learning_rate': 0.04936231876523339, 'colsample_bytree': 0.9994140001657239, 'colsample_bynode': 0.8348193255981815, 'lambda_l1': 2.8478357976652857, 'lambda_l2': 1.3445067021941788, 'max_depth': 13}. Best is trial 81 with value: 24.675671423250755.
[LightGBM] [Warning] lambda_l1 is set=2.271135770923234, reg_alpha=0.0 will be ignored. Current value: lambda_l1=2.271135770923234
[LightGBM] [Warning] lambda_l2 is set=0.8029636530217137, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.8029636530217137
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] lambda_l1 is set=2.271135770923234, reg_alpha=0.0 will be ignored. Current value: lambda_l1=2.271135770923234
[LightGBM] [Warning] lambda_l2 is set=0.8029636530217137, reg_lambda=0.0 will be ignored. Current value: lam

Best trial: 81. Best value: 24.6757: 100%|██████████| 100/100 [2:27:10<00:00, 88.30s/it]

[I 2023-12-21 21:36:49,367] Trial 99 finished with value: 25.1542115834817 and parameters: {'n_estimators': 1449, 'learning_rate': 0.09067426341537975, 'colsample_bytree': 0.9811148056004526, 'colsample_bynode': 0.7819540190760144, 'lambda_l1': 2.271135770923234, 'lambda_l2': 0.8029636530217137, 'max_depth': 12}. Best is trial 81 with value: 24.675671423250755.
Number of finished trials: 100
Best trial: {'n_estimators': 1475, 'learning_rate': 0.09994117288591162, 'colsample_bytree': 0.9967406569464257, 'colsample_bynode': 0.8154020931521416, 'lambda_l1': 4.459227677749405, 'lambda_l2': 0.6607768224314803, 'max_depth': 14}


# Model

In [10]:
# Let's build separate models for produciton and consumption of electricity
model_parameters = {
    'n_estimators': 1000,
    'learning_rate': 0.05,
    'colsample_bytree': 0.9,
    'colsample_bynode': 0.6,
    'lambda_l1': 3.5,
    'lambda_l2': 1.5,
    'max_depth': 12
}

# standard technique to use mean predictions from several models with different random_state
model_consumption = VotingRegressor([
    (
        f'consumption_lgb_{i}', 
         lgb.LGBMRegressor(**model_parameters, random_state=i)
    ) for i in range(10)
])
model_production = VotingRegressor([
    (
        f'production_lgb_{i}', 
         lgb.LGBMRegressor(**model_parameters, random_state=i)
    ) for i in range(10)
])


        
mask = df_train_features['is_consumption'] == 1
model_consumption.fit(
    X=df_train_features[mask].drop(columns=["target"]),
    y=df_train_features[mask]["target"]
)

mask = df_train_features['is_consumption'] == 0
model_production.fit(
    X=df_train_features[mask].drop(columns=["target"]),
    y=df_train_features[mask]["target"]
)

[LightGBM] [Warning] lambda_l2 is set=1.5, reg_lambda=0.0 will be ignored. Current value: lambda_l2=1.5
[LightGBM] [Warning] lambda_l1 is set=3.5, reg_alpha=0.0 will be ignored. Current value: lambda_l1=3.5
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] lambda_l2 is set=1.5, reg_lambda=0.0 will be ignored. Current value: lambda_l2=1.5
[LightGBM] [Warning] lambda_l1 is set=3.5, reg_alpha=0.0 will be ignored. Current value: lambda_l1=3.5
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.094530 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 31740
[LightGBM] [Info] Number of data points in the train set: 825951,

KeyboardInterrupt: 

# Submit API

In [ ]:
import enefit

env = enefit.make_env()
iter_test = env.iter_test()

The following code demonstrated usage of API when in each cycle step we simulate new day and we need to send prediciotns before we get the next day (this guranteeres that we don't see targets from future).

Local running of a notebook uses toy example of test data, after notebook submission toy data will be replaced with new unseen data.

In [ ]:
for (
    test, 
    revealed_targets, 
    client, 
    historical_weather,
    forecast_weather, 
    electricity_prices, 
    gas_prices, 
    sample_prediction
) in iter_test:
    
    test = test.rename(columns={"prediction_datetime": "datetime"})
    
    df_test = pl.from_pandas(test[data_cols[1:]], schema_overrides=schema_data)
    df_client = pl.from_pandas(client[client_cols], schema_overrides=schema_client)
    df_gas_prices = pl.from_pandas(gas_prices[gas_prices_cols], schema_overrides=schema_gas)
    df_electricity_prices = pl.from_pandas(electricity_prices[electricity_prices_cols], schema_overrides=schema_electricity)
    df_new_forecast_weather = pl.from_pandas(forecast_weather[forecast_weather_cols], schema_overrides=schema_forecast)
    df_new_historical_weather = pl.from_pandas(historical_weather[historical_weather_cols], schema_overrides=schema_historical)
    df_new_target = pl.from_pandas(revealed_targets[target_cols], schema_overrides=schema_target)
    
    df_forecast_weather = pl.concat([df_forecast_weather, df_new_forecast_weather]).unique(['forecast_datetime', 'latitude', 'longitude', 'hours_ahead'])
    df_historical_weather = pl.concat([df_historical_weather, df_new_historical_weather]).unique(['datetime', 'latitude', 'longitude'])
    df_target = pl.concat([df_target, df_new_target]).unique(['datetime', 'county', 'is_business', 'product_type', 'is_consumption'])
    
    df_test_features = generate_features(
        df_test, 
        df_client, 
        df_gas_prices, 
        df_electricity_prices, 
        df_forecast_weather, 
        df_historical_weather, 
        df_weather_station_to_county_mapping, 
        df_target
    )
    df_test_features = to_pandas(df_test_features)
    
    mask = df_test_features['is_consumption'] == 1
    # clip method makes values < 0 equal 0 because our target is nonnegative and models can produce negative values
    sample_prediction.loc[mask.values, "target"] = model_consumption.predict(df_test_features[mask]).clip(0)
    
    mask = df_test_features['is_consumption'] == 0
    sample_prediction.loc[mask.values, "target"] = model_production.predict(df_test_features[mask]).clip(0)
    
    # send predictions
    env.predict(sample_prediction)